In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra  
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
bureau_balance=pd.read_csv('/kaggle/input/home-credit-default-risk/bureau_balance.csv')
cash=pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv')
client=pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
HomeCredit=pd.read_csv('/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv')
test=pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
previous=pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')
credit=pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')
installements=pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')
bureau=pd.read_csv('/kaggle/input/home-credit-default-risk/bureau.csv')

In [ ]:
HomeCredit[HomeCredit["Table"]=="application_{train|test}.csv"][HomeCredit["Row"]=="EXT_SOURCE_3"]['Description']

**Sampling**

In [ ]:
import random 
L=list(np.arange(len(client)))
randomList=random.sample(L,int(len(L)/10))
data=client.iloc[np.setdiff1d(list(client.index),randomList)]
indice=list(data[data['TARGET']==1].index)
indice=random.sample(indice,int(len(indice)/10))
randomList=randomList+indice
sample_client=client.iloc[randomList,:].reset_index()
sample_client.drop('index',axis=1,inplace=True)

In [ ]:
#L=list(np.arange(len(test)))
#randomList=random.sample(L,int(len(L)/10))
#sample_test=test.iloc[randomList,:].reset_index()
#sample_test.drop('index',axis=1,inplace=True)

In [ ]:
test['TARGET']=np.nan
test['set']='test'
sample_client['set']='train'
sample=sample_client.append(test,ignore_index=True)

In [ ]:
len(sample_client[sample_client['TARGET']==1])/len(sample_client[sample_client['TARGET']==0])

In [ ]:
len(client[client['TARGET']==1])/len(client[client['TARGET']==0])

In [ ]:
def sampling(data_father,data_child,col):
    ID=data_father[col].values
    sample_data=data_child[data_child[col].isin(ID)].reset_index()
    sample_data.drop('index',axis=1,inplace=True)
    return sample_data

In [ ]:
sample_bureau=sampling(sample,bureau,'SK_ID_CURR')

In [ ]:
sample_bureau['SK_ID_CURR'].unique().sort()==sample['SK_ID_CURR'].unique().sort()

In [ ]:
sample_bureau_balance=sampling(sample_bureau,bureau_balance,'SK_ID_BUREAU')

In [ ]:
sample_previous=sampling(sample,previous,'SK_ID_CURR')

In [ ]:
sample_cash=sampling(sample_previous,cash,'SK_ID_PREV')

In [ ]:
sample_credit=sampling(sample_previous,credit,'SK_ID_PREV')

In [ ]:
sample_installements=sampling(sample_previous,installements,'SK_ID_PREV')

In [ ]:
def replace_day_outliers(df):
    for col in df.columns:
        if "DAYS " in col:
            df[col]=df[col].replace({365243:np.nan})

In [ ]:
sample=replace_day_outliers(sample)

In [ ]:
sample_bureau=replace_day_outliers(sample_bureau)
sample_bureau_balance=replace_day_outliers(sample_bureau_balance)
sample_previous=replace_day_outliers(sample_previous)
sample_credit=replace_day_outliers(sample_credit)
sample_cash=replace_day_outliers(sample_cash)
sample_installements=replace_day_outliers(sample_installements)

In [ ]:
start_date=pd.Timestamp("2021/07/10")

In [ ]:
for col in ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']:
    sample_bureau[col] = pd.to_timedelta(sample_bureau[col], 'D')

In [ ]:
sample_bureau[['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']].head()

In [ ]:
sample_bureau['bureau_credit_application_date'] = start_date + sample_bureau['DAYS_CREDIT']
sample_bureau['bureau_credit_end_date'] = start_date + sample_bureau['DAYS_CREDIT_ENDDATE']
sample_bureau['bureau_credit_close_date'] = start_date + sample_bureau['DAYS_ENDDATE_FACT']
sample_bureau['bureau_credit_update_date'] = start_date + sample_bureau['DAYS_CREDIT_UPDATE']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Set up default plot styles
plt.rcParams['font.size'] = 26
plt.style.use('fivethirtyeight')

# Drop the time offset columns
#sample_bureau = sample_bureau.drop(columns = ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE'])
plt.figure(figsize = (10, 8))
sns.distplot((sample_bureau['bureau_credit_end_date'] - sample_bureau['bureau_credit_application_date']).dropna().dt.days);
plt.xlabel('Length of Loan (Days)', size = 24); plt.ylabel('Density', size = 24); plt.title('Loan Length', size = 30);

In [ ]:
sample_bureau_balance['SK_ID_BUREAU'].unique()

In [ ]:
# Convert to timedelta
sample_bureau_balance['MONTHS_BALANCE'] = pd.to_timedelta(sample_bureau_balance['MONTHS_BALANCE'], 'M')
# Make a date column
sample_bureau_balance['bureau_balance_date'] = start_date + sample_bureau_balance['MONTHS_BALANCE']
sample_bureau_balance = sample_bureau_balance.drop(columns = ['MONTHS_BALANCE'])

# Select one loan and plot
example_credit = sample_bureau_balance[sample_bureau_balance['SK_ID_BUREAU'] ==  5718492]
plt.plot(example_credit['bureau_balance_date'], example_credit['STATUS'], 'ro');
plt.title('Loan 5001709 over Time'); plt.xlabel('Date'); plt.ylabel('Status');



In [ ]:
for col in sample.columns:
    print(col)
    ,sample[col].max(),sample[col].min()

**Work with Dask and Featuretools made**

In [ ]:
pip install dask

In [ ]:
pip install "dask[dataframe]"

In [ ]:
pip install featuretools

In [ ]:
import featuretools as ft 

In [ ]:
es_pandas=ft.EntitySet(id='client_pandas')
es=ft.EntitySet(id='clients')

In [ ]:
import dask.dataframe as dd

In [ ]:
sample_dask=dd.from_pandas(sample,4)
sample_bureau_dask=dd.from_pandas(sample_bureau,4)
sample_bureau_balance_dask=dd.from_pandas(sample_bureau_balance,4)
sample_previous_dask=dd.from_pandas(sample_previous,4)
sample_cash_dask=dd.from_pandas(sample_cash,4)
sample_credit_dask=dd.from_pandas(sample_credit,4)
sample_installements_dask=dd.from_pandas(sample_installements,4)


In [ ]:
dic={}
L=['set','TARGET','SK_ID_CURR','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_CAR','FLAG_OWN_REALTY','EMERGENCYSTATE_MODE','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY']
dic['SK_ID_CURR']=ft.variable_types.Id
dic['CODE_GENDER']=ft.variable_types.Boolean
dic['FLAG_OWN_CAR']=ft.variable_types.Boolean
dic['FLAG_OWN_REALTY']=ft.variable_types.Boolean
dic['EMERGENCYSTATE_MODE']=ft.variable_types.Boolean
dic['REGION_RATING_CLIENT']=ft.variable_types.Ordinal
dic['REGION_RATING_CLIENT_W_CITY']=ft.variable_types.Ordinal
dic['TARGET']=ft.variable_types.Boolean
dic['set']=ft.variable_types.Boolean
L.append('SK_ID_CURR')
for col in sample.columns:
    if(col not in L):
        if(sample[col].dtype=='O'):
            dic[col]=ft.variable_types.Categorical
        elif(sample[col].dtype=='int64'):
            if(len(sample[col].unique())==2):
                 dic[col]=ft.variable_types.Boolean
            else:
                dic[col]=ft.variable_types.Numeric
        else:
                dic[col]=ft.variable_types.Numeric


In [ ]:
es=es.entity_from_dataframe(entity_id='sample',dataframe=sample_dask,index='SK_ID_CURR',variable_types=dic)
es_pandas=es_pandas.entity_from_dataframe(entity_id='sample',dataframe=sample,index='SK_ID_CURR',variable_types=dic)

In [ ]:
dic={}
dic['SK_ID_CURR']=ft.variable_types.Id
dic['SK_ID_BUREAU']=ft.variable_types.Id
L=['SK_ID_CURR','SK_ID_BUREAU']
for col in sample_bureau.columns:
        if(col not in L):
            if(sample_bureau[col].dtype=='O'):
                dic[col]=ft.variable_types.Categorical
            elif(sample_bureau[col].dtype=='int64'):
                if(len(sample_bureau[col].unique())==2):
                    dic[col]=ft.variable_types.Boolean
                else:
                    dic[col]=ft.variable_types.Numeric
            else:
                dic[col]=ft.variable_types.Numeric

In [ ]:
es=es.entity_from_dataframe(entity_id='sample_bureau',dataframe=sample_bureau_dask,index='SK_ID_BUREAU',variable_types=dic)
es_pandas=es_pandas.entity_from_dataframe(entity_id='sample_bureau',dataframe=sample_bureau,index='SK_ID_BUREAU',variable_types=dic)

In [ ]:
dic={'SK_ID_BUREAU':ft.variable_types.Id,'MONTHS_BALANCE':ft.variable_types.Numeric,'STATUS':ft.variable_types.Categorical}

In [ ]:
es=es.entity_from_dataframe(entity_id='sample_bureau_balance',dataframe=sample_bureau_balance_dask,make_index=True,index = 'bureaubalance_index',variable_types=dic)

In [ ]:
es_pandas=es_pandas.entity_from_dataframe(entity_id='sample_bureau_balance',dataframe=sample_bureau_balance,index = 'bureaubalance_index',variable_types=dic)

In [ ]:
dic={}
dic['SK_ID_PREV']=ft.variable_types.Id
dic['SK_ID_CURR']=ft.variable_types.Id
dic['FLAG_LAST_APPL_PER_CONTRACT']=ft.variable_types.Boolean
dic['NAME_YIELD_GROUP']=ft.variable_types.Ordinal
dic['NFLAG_INSURED_ON_APPROVAL']=ft.variable_types.Boolean
L=['SK_ID_PREV','SK_ID_CURR','FLAG_LAST_APPL_PER_CONTRACT','NAME_YIELD_GROUP','NFLAG_INSURED_ON_APPROVAL']
for col in sample_previous.columns:
        if(col not in L):
            if(sample_previous[col].dtype=='O'):
                dic[col]=ft.variable_types.Categorical
            elif(sample_previous[col].dtype=='int64'):
                if(len(sample_previous[col].unique())==2):
                    dic[col]=ft.variable_types.Boolean
                else:
                    dic[col]=ft.variable_types.Numeric
            else:
                dic[col]=ft.variable_types.Numeric

In [ ]:
es=es.entity_from_dataframe(entity_id='sample_previous',dataframe=sample_previous_dask,index = 'SK_ID_PREV',variable_types=dic)
es_pandas=es_pandas.entity_from_dataframe(entity_id='sample_previous',dataframe=sample_previous,index = 'SK_ID_PREV',variable_types=dic)

In [ ]:
dic={'SK_ID_PREV':ft.variable_types.Id,'SK_ID_CURR':ft.variable_types.Id,'MONTHS_BALANCE':ft.variable_types.Numeric,'CNT_INSTALMENT':ft.variable_types.Numeric,'CNT_INSTALMENT_FUTURE':ft.variable_types.Numeric,'NAME_CONTRACT_STATUS':ft.variable_types.Categorical,'SK_DPD':ft.variable_types.Id,'SK_DPD_DEF':ft.variable_types.Id}

In [ ]:
es = es.entity_from_dataframe(entity_id = 'sample_cash', dataframe = sample_cash_dask,make_index=True,index="cash_id",variable_types=dic)


In [ ]:
es_pandas = es_pandas.entity_from_dataframe(entity_id = 'sample_cash', dataframe = sample_cash,make_index=True,index="cash_id",variable_types=dic)

In [ ]:
dic={}
dic['SK_ID_PREV']=ft.variable_types.Id
dic['SK_ID_CURR']=ft.variable_types.Id
dic['SK_DPD']=ft.variable_types.Id
dic['SK_DPD_DEF']=ft.variable_types.Id
L=['SK_ID_PREV','SK_ID_CURR','SK_DPD','SK_DPD_DEF']
for col in sample_credit.columns:
        if(col not in L):
            if(sample_credit[col].dtype=='O'):
                dic[col]=ft.variable_types.Categorical
            elif(sample_credit[col].dtype=='int64'):
                if(len(sample_credit[col].unique())==2):
                    dic[col]=ft.variable_types.Boolean
                else:
                    dic[col]=ft.variable_types.Numeric
            else:
                dic[col]=ft.variable_types.Numeric

In [ ]:
es = es.entity_from_dataframe(entity_id = 'sample_credit', dataframe = sample_credit_dask,make_index=True,index="credit_id",variable_types=dic)


In [ ]:
es_pandas = es_pandas.entity_from_dataframe(entity_id = 'sample_credit', dataframe = sample_credit,make_index=True,index="credit_id",variable_types=dic)

In [ ]:
dic={}
dic['SK_ID_PREV']=ft.variable_types.Id
dic['SK_ID_CURR']=ft.variable_types.Id
L=['SK_ID_PREV','SK_ID_CURR']
for col in sample_installements.columns:
        if(col not in L):
            if(sample_installements[col].dtype=='O'):
                dic[col]=ft.variable_types.Categorical
            elif(sample_installements[col].dtype=='int64'):
                if(len(sample_installements[col].unique())==2):
                    dic[col]=ft.variable_types.Boolean
                else:
                    dic[col]=ft.variable_types.Numeric
            else:
                dic[col]=ft.variable_types.Numeric

In [ ]:
es = es.entity_from_dataframe(entity_id = 'sample_installements', dataframe = sample_installements_dask,make_index=True,index="installements_id",variable_types=dic)


In [ ]:
es_pandas = es_pandas.entity_from_dataframe(entity_id = 'sample_installements', dataframe = sample_installements,make_index=True,index="installements_id",variable_types=dic)

In [ ]:
es

In [ ]:
es_pandas

In [ ]:
# Relationship between app and bureau
r_app_bureau = ft.Relationship(es['sample']['SK_ID_CURR'], es['sample_bureau']['SK_ID_CURR'])

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['sample_bureau']['SK_ID_BUREAU'], es['sample_bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['sample']['SK_ID_CURR'], es['sample_previous']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['sample_previous']['SK_ID_PREV'], es['sample_cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['sample_previous']['SK_ID_PREV'], es['sample_installements']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['sample_previous']['SK_ID_PREV'], es['sample_credit']['SK_ID_PREV'])

In [ ]:
es_pandas = es_pandas.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])

In [ ]:
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])

In [ ]:
    feature_matrix_spec_pandas, feature_names_spec_pandas = ft.dfs(entityset = es_pandas, target_entity = 'sample',  
                                                     agg_primitives = ['sum', 'count', 'min', 'max', 'mean'], 
                                                     max_depth = 2, features_only = False, verbose = True)

In [ ]:
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, target_entity = 'sample',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean'], 
                                                 max_depth = 2, features_only = False, verbose = True)

In [ ]:
#rest=np.setdiff1d(cols_to_remove,missing_columns)

**Explanatory Data** 

In [ ]:
sample_train=feature_matrix_spec[feature_matrix_spec['set']=='train'].compute()

In [ ]:
sample_test=feature_matrix_spec[feature_matrix_spec['set']=='test'].compute()

In [ ]:
sample_train.to_csv('ssample_train.csv', index=False)
sample_test.to_csv('ssample_test.csv', index=False)

In [ ]:
def missing_value(train,alpha):
    train_miss_value=pd.DataFrame(train.isna().sum())
    train_miss_value['percentage']=train_miss_value/len(train)
    train_miss_value=train_miss_value.sort_values('percentage',ascending=False)
    return list(train_miss_value[train_miss_value['percentage']>alpha].index)
    

In [ ]:
L=missing_value(sample_train.compute(),0.6)

In [ ]:
L_test=missing_value(sample_test.compute(),0.6)

In [ ]:
missing_columns=list(set(L+L_test))

In [ ]:
missing_columns.remove("TARGET")

In [ ]:
sample_train=sample_train.drop(columns=['set'])

In [ ]:
sample_test=sample_test.drop(columns=['set','TARGET'])

In [ ]:
#train, test = sample_train.align(sample_test, join = 'inner', axis = 1)

In [ ]:
sample_train=sample_train.drop(columns=missing_columns)
sample_test=sample_test.drop(columns=missing_columns)

**Correlation**

In [ ]:
correlation=sample_train.corr()

In [ ]:
correlation_pandas=correlation.compute()

In [ ]:
correlations_target=correlation_pandas['TARGET'].dropna()

In [ ]:
corr_series=correlations_target.sort_values()

In [ ]:
corr_series.sort_values().head()

In [ ]:
corr_series.sort_values().tail()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def kde_target(var_name, df):
    corr = df['TARGET'].corr(df[var_name])
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    plt.figure(figsize = (12, 6))
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [ ]:
kde_target('MEAN(sample_bureau.DAYS_CREDIT)',feature_matrix_spec_pandas)

In [ ]:
kde_target('EXT_SOURCE_3',feature_matrix_spec_pandas)

In [ ]:
threshold=0.8
above_threshold_vars = {}
for col in correlation_pandas:
    above_threshold_vars[col] = list(correlation_pandas.index[correlation_pandas[col] > threshold])

In [ ]:
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

# Iterate through columns and correlated columns
for key, value in above_threshold_vars.items():
    # Keep track of columns already examined
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            # Only want to remove one in a pair
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
            
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

In [ ]:
sample_train=sample_train.drop(columns=cols_to_remove)
sample_test=sample_test.drop(columns=cols_to_remove)

In [ ]:
sample_train.select_dtypes('object').columns

In [ ]:
def countplot_withY(label, dataset):
  plt.figure(figsize=(10,5))
  Y = dataset[label]
  total = len(Y)*1.
  ax=sns.countplot(x=label, data=dataset, hue="TARGET")
  for p in ax.patches:
    ax.annotate('{:.1f}%'.format(100*p.get_height()/total), (p.get_x()+0.1, p.get_height()+5))

  #put 11 ticks (therefore 10 steps), from 0 to the total number of rows in the dataframe
  ax.yaxis.set_ticks(np.linspace(0, total, 11))
  #adjust the ticklabel to the desired format, without changing the position of the ticks.
  ax.set_yticklabels(map('{:.1f}%'.format, 100*ax.yaxis.get_majorticklocs()/total))
  ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
  # ax.legend(labels=["no","yes"])
  plt.show()

In [ ]:
sample_train_pandas=feature_matrix_spec_pandas[feature_matrix_spec_pandas['set']=='train']

In [ ]:
countplot_withY('CODE_GENDER',sample_train_pandas)

In [ ]:
countplot_withY('OCCUPATION_TYPE',sample_train_pandas)

In [ ]:
countplot_withY('FLAG_OWN_CAR',sample_train_pandas)

**Modeling**

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import gc
import matplotlib.pyplot as plt

In [ ]:
pip install optuna 

In [ ]:
import optuna 

In [ ]:
k_fold = KFold(n_splits =5, shuffle = True, random_state = 50)

In [ ]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    #features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    features = features.drop(columns = ['TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    param_grid={"n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),"reg_alpha":trial.suggest_float("reg_alpha",0.05,0.2),"reg_lambda":trial.suggest_float("reg_lambda",0.05,0.1)}
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        """model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.01, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)"""
        model=lgb.LGBMClassifier(objective="binary",subsample=0.8,n_jobs=-1,random_state=50,class_weight="balanced",**param_grid)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [ ]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    print(df['feature'].head(15).values)
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
sample_train_pandas.reset_index(inplace=True)
sample_train_pandas=sample_train_pandas[sample_train.columns]

In [ ]:
sample_test_pandas=feature_matrix_spec_pandas[feature_matrix_spec_pandas['set']=='test']

In [ ]:
sample_test_pandas.reset_index(inplace=True)
sample_test_pandas=sample_test_pandas[sample_test.columns]

In [ ]:
submission, fi, metrics = model(sample_train_pandas, sample_test_pandas)

In [ ]:
metrics

In [ ]:
submission.to_csv('submission2.csv', index=False)

In [ ]:
sample_client.reset_index(inplace=True)

In [ ]:
sample_client.drop('index',axis=1,inplace=True)

In [ ]:
pd.value_counts(sample_client['TARGET'])

In [ ]:
sample_previous_2=sampling(sample_client,previous,'SK_ID_CURR')

In [ ]:
sample_cash_2=sampling(sample_previous_2,cash,'SK_ID_PREV')
sample_bureau_2=sampling(sample_client,bureau,'SK_ID_CURR')

In [ ]:
data1=sample_previous_2.groupby('SK_ID_CURR')['RATE_DOWN_PAYMENT'].max()

In [ ]:
data1

In [ ]:
sample_client=sample_client.merge(data1,on='SK_ID_CURR',how='left')

In [ ]:
data2=sample_bureau_2.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].max()
sample_client=sample_client.merge(data2,on='SK_ID_CURR',how='left')

In [ ]:
data3=sample_previous_2.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].max()
sample_client=sample_client.merge(data3,on='SK_ID_CURR',how='left')

In [ ]:
data4=sample_bureau_2.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].max()
sample_client=sample_client.merge(data4,on='SK_ID_CURR',how='left')

In [ ]:
X_prime=sample_client[['SK_ID_CURR','CODE_GENDER','RATE_DOWN_PAYMENT','DAYS_CREDIT_ENDDATE','AMT_DOWN_PAYMENT','DAYS_ENDDATE_FACT','EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'DAYS_BIRTH', 'AMT_CREDIT',
 'DAYS_EMPLOYED', 'AMT_ANNUITY' ,'DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE','YEARS_BEGINEXPLUATATION_AVG','TARGET']]
X_prime.rename(columns={'RATE_DOWN_PAYMENT':'MAX(sample_previous.RATE_DOWN_PAYMENT)','DAYS_CREDIT_ENDDATE':'MAX(sample_bureau.DAYS_CREDIT_ENDDATE)','AMT_DOWN_PAYMENT':'MAX(sample_previous.AMT_DOWN_PAYMENT)','DAYS_ENDDATE_FACT':'MAX(sample_bureau.DAYS_ENDDATE_FACT)'},inplace=True)

In [ ]:
fi_sorted = plot_feature_importances(fi)

In [ ]:
submission, fi, metrics = model(X_prime, sample_test_pandas[X_prime.columns[:-1]])

In [ ]:
metrics

In [ ]:
submission.to_csv('submission3.csv',index=False)

In [ ]:
sample_train_pandas.select_dtypes('object').columns

In [ ]:
pd.value_counts(sample_train_pandas['TARGET']).plot.bar()
plt.title('Fraud class histogram')
plt.xlabel('TARGET')
plt.ylabel('Frequency')
sample_train_pandas['TARGET'].value_counts()

**SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
data=pd.DataFrame(sample_train_pandas.isna().sum())
L=list(data[data[0]==0].index)

In [ ]:
list(sample_train_pandas.select_dtypes('object'))

In [ ]:
L=list(np.setdiff1d(L,list(sample_train_pandas.select_dtypes('object').columns)))

In [ ]:
cat=['CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_CONTRACT_TYPE',

 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',

 ]

##### 

In [ ]:
X=sample_train_pandas[L]

In [ ]:
smote = SMOTE(random_state = 101)
X_oversample, y_oversample = smote.fit_resample(X, X['TARGET'])

In [ ]:
pd.value_counts(X_oversample['TARGET']).plot.bar()
plt.title('Fraud class histogram')
plt.xlabel('TARGET')
plt.ylabel('Frequency')
X_oversample['TARGET'].value_counts()

In [ ]:
L=list(X_oversample.columns)

In [ ]:
L.append('SK_ID_CURR')

In [ ]:
submission, fi, metrics = model(X_oversample, sample_test_pandas[L])

In [ ]:
submission=submission.reset_index()

In [ ]:
#submission.to_csv('submission.csv', index=False)

In [ ]:
sns.scatterplot(x=sample_train_pandas.index,y='DAYS_BIRTH',hue='TARGET',data=sample_train_pandas)

In [ ]:
'EXT_SOURCE_3' in sample_train_pandas.columns

In [ ]:
sample_train_pandas